In [6]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, '/global/homes/d/dbrookes/design_icml/')
import itertools
from keras.layers import Input, Dense, Reshape, Flatten
from keras import layers, initializers
from keras.models import Model, load_model
import keras.backend as K
import numpy as np
from seqtools import SequenceTools as ST
from gfp_gp import SequenceGP
from util import AA, AA_IDX
from util import build_vae
from sklearn.model_selection import train_test_split, ShuffleSplit
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
from gan import WGAN
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import scipy.stats
from scipy.stats import norm

from scipy.optimize import minimize
from keras.utils.generic_utils import get_custom_objects
from util import one_hot_encode_aa, partition_data, get_balaji_predictions, get_samples
from util import convert_idx_array_to_aas, build_pred_vae_model, get_experimental_X_y
from util import get_gfp_X_y_aa
from losses import neg_log_likelihood
import json

In [7]:
def bombarelli_opt(X_train, pred_vae, ground_truth, total_it=10000, constrained = True):
    LD =20
    L = X_train.shape[1]
    
    embeddings = pred_vae.encoder_.predict([X_train])[0]
    predictions = pred_vae.predictor_.predict(embeddings)[0]
    predictions = predictions.reshape((predictions.shape[0]))
    
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    
#     embeddings = embeddings[:100]
#     predictions = predictions[:100]
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, normalize_y=True)
    print("Fitting GP...")
    gp.fit(embeddings,predictions)
    print("Finishing fitting GP")
    
    def gp_z_estimator(z):
        val = gp.predict(z.reshape(1,-1)).ravel()[0]
        return -val 

    method='COBYLA'
    num_it = 0
    f_opts = []
    gt_opts = []
    
    z0 = np.random.randn(LD)
    if constrained:
        z_norm = np.linalg.norm(embeddings, axis=1)
        z_norm_mean = np.mean(z_norm)
        z_norm_std = np.std(z_norm)

        lower_rad = z_norm_mean - z_norm_std*2
        higher_rad = z_norm_mean + z_norm_std*2
        constraints = [{'type': 'ineq', 'fun': lambda x:  np.linalg.norm(x)-lower_rad },
                   {'type': 'ineq', 'fun': lambda x:  higher_rad-np.linalg.norm(x) }]
        res = minimize(gp_z_estimator, z0, method=method, constraints=constraints,
                       options={'disp': False,'tol':0.1,'maxiter':10000},tol=0.1)
    else:
        res = minimize(gp_z_estimator, z0, method=method,
                       options={'disp': False,'tol':0.1,'maxiter':10000},tol=0.1)
    z_opt = res.x.reshape((1, LD))
    f_opt = -res.fun
#         num_it += res.nfev
    
    x_opt = np.argmax(pred_vae.decoder_.predict(z_opt), axis=-1)
    gt_opt = ground_truth.predict(x_opt)[:, 0][0]
#     f_opts.append(f_opt)
#     gt_opts.append(gt_opts)
    print(f_opt, gt_opt)
    
    oracle_max_seq = convert_idx_array_to_aas(x_opt)
    
    max_dict = {'oracle_max' : f_opt, 
                'oracle_max_seq': oracle_max_seq, 
                'gt_of_oracle_max': gt_opt}

    results = np.array([f_opt, gt_opt])
    return results, max_dict

In [8]:
def train_experimental_pred_vaes():
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    for it in range(3):
        RANDOM_STATE = it + 1
        X_train, y_train, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
        
        L = X_train.shape[1]
        LD=20
        gt_var=0.01
        pred_vae = build_pred_vae_model(latent_dim=LD, n_tokens=X_train.shape[2], 
                                    seq_length=L, enc1_units=50, pred_var=gt_var)

        pred_vae.fit([X_train], [X_train, np.zeros(X_train.shape[0]), y_train, np.zeros_like(y_train)],
                      batch_size=10,
                      epochs=100,
                      shuffle=True,
                      validation_split=0,
                      verbose=2
                    )
        suffix = "_%s_%i" % (train_size_str, RANDOM_STATE)
        pred_vae.encoder_.save_weights("models/pred_vae_encoder_weights%s.h5" % suffix)
        pred_vae.decoder_.save_weights("models/pred_vae_decoder_weights%s.h5" % suffix)
        pred_vae.predictor_.save_weights("models/pred_vae_predictor_weights%s.h5" % suffix)
        pred_vae.vae_.save_weights("models/pred_vae_vae_weights%s.h5" % suffix)

In [12]:
def run_bombarelli(constrained=True):
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    for it in range(3):
        RANDOM_STATE = it + 1
        
        X_train, _, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
        ground_truth = SequenceGP(load=True, load_prefix="data/gfp_gp")
        
        L = X_train.shape[1]
        LD=20
        gt_var=0.01
        pred_vae = build_pred_vae_model(latent_dim=LD, n_tokens=X_train.shape[2], 
                                    seq_length=L, enc1_units=50, pred_var=gt_var)
        suffix = "_%s_%i" % (train_size_str, RANDOM_STATE)
        
        pred_vae.encoder_.load_weights("models/pred_vae_encoder_weights%s.h5" % suffix)
        pred_vae.decoder_.load_weights("models/pred_vae_decoder_weights%s.h5" % suffix)
        pred_vae.predictor_.load_weights("models/pred_vae_predictor_weights%s.h5" % suffix)
        pred_vae.vae_.load_weights("models/pred_vae_vae_weights%s.h5" % suffix)
        if not constrained:
            suffix = "_unconstrained" + suffix
        bomb_results, test_max = bombarelli_opt(X_train, pred_vae, ground_truth, total_it=1000, constrained=constrained)
#         np.save("results/bombarelli_results_unconstrained%s.npy" % suffix, bomb_results)
        with open('results/%s_max%s.json'% ('bombarelli', suffix), 'w') as outfile:
            json.dump(test_max, outfile)

In [13]:
# train_experimental_pred_vaes()
# run_bombarelli(constrained=True)
run_bombarelli(constrained=False)

Fitting GP...
Finishing fitting GP
4.979642530364799 3.2330937525428127
Fitting GP...
Finishing fitting GP
3.3658104483001807 3.0088897061997057
Fitting GP...
Finishing fitting GP
3.4049551071075257 3.0677834255280723
